In [27]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from sklearn.model_selection import StratifiedGroupKFold
from sklearn.metrics import precision_recall_curve, auc, accuracy_score
from sklearn.preprocessing import MinMaxScaler
from datetime import timedelta
from xgboost import XGBClassifier

from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

import kaggle

from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()

In [28]:
full_train_df = pd.read_csv('obs_train.csv')
full_test_df = pd.read_csv('obs_test.csv')

full_y = full_train_df['SepsisLabel']
full_X = full_train_df.drop(columns = ['SepsisLabel', 'person_id', 'datetime'])
test_labels = full_test_df[['person_id', 'datetime']].copy()
full_test = full_test_df.drop(columns = ['person_id', 'datetime'])

mmx = MinMaxScaler()
full_X = mmx.fit_transform(full_X)
full_test = mmx.transform(full_test)

print(full_train_df.shape)
full_train_df.info(verbose=True)

(331637, 263)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 331637 entries, 0 to 331636
Data columns (total 263 columns):
 #    Column                                                    Dtype  
---   ------                                                    -----  
 0    person_id                                                 int64  
 1    SepsisLabel                                               int64  
 2    datetime                                                  object 
 3    datetime_year                                             int64  
 4    datetime_month                                            int64  
 5    datetime_day                                              int64  
 6    datetime_hour                                             int64  
 7    datetime_weekday                                          int64  
 8    MALE                                                      float64
 9    age_in_months                                             float64
 10   hour

In [29]:
train_splits = []
test_splits = []

sgkf = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=42)

for train_idx, test_idx in sgkf.split(full_train_df, full_train_df['SepsisLabel'], groups=full_train_df['person_id']):
    train_df = full_train_df.iloc[train_idx].copy()
    test_df = full_train_df.iloc[test_idx].copy()

    train_df.drop(columns=['person_id', 'datetime'], inplace=True)
    test_df.drop(columns=['person_id', 'datetime'], inplace=True)

    train_y = train_df['SepsisLabel']
    train_X = train_df.drop(columns='SepsisLabel')
    test_y = test_df['SepsisLabel']
    test_X = test_df.drop(columns='SepsisLabel')

    mmx = MinMaxScaler()
    train_X = mmx.fit_transform(train_X)
    test_X = mmx.transform(test_X)

    train_splits.append((train_X, train_y))
    test_splits.append((test_X, test_y))

In [30]:
params = {
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1.0),  # Feature subsampling
    'gamma': hp.uniform('gamma', 0, 2),  # Higher values prevent overfitting
    'learning_rate': hp.loguniform('learning_rate', -5.0, -2.3),  # Log scale for better tuning
    'max_depth': hp.quniform('max_depth', 3, 12, 1),  # Deeper trees may be needed
    'min_child_weight': hp.loguniform('min_child_weight', 0, 3),  # Prevents small leaves
    'n_estimators': hp.quniform('n_estimators', 100, 500, 50),  # More trees for stability
    'reg_alpha': hp.loguniform('reg_alpha', -3, 2),  # L1 regularization (sparse features)
    'reg_lambda': hp.loguniform('reg_lambda', -3, 2),  # L2 regularization
    'subsample': hp.uniform('subsample', 0.5, 1.0),  # Row subsampling
}

In [31]:
def objective(params):
    clf = XGBClassifier(
        n_estimators=int(params['n_estimators']),
        learning_rate=params['learning_rate'],
        max_depth=int(params['max_depth']),
        min_child_weight=params['min_child_weight'],
        subsample=params['subsample'],
        colsample_bytree=params['colsample_bytree'],
        gamma=params['gamma'],
        reg_alpha=params['reg_alpha'],
        reg_lambda=params['reg_lambda'],
        objective='binary:logistic',
        eval_metric='aucpr',
        random_state=0
    )
    
    praucs_test = []
    praucs_train =[]

    for ((train_X, train_y), (test_X, test_y)) in zip(train_splits, test_splits):
        eval = [(train_X, train_y), (test_X, test_y)]
        clf.fit(train_X, train_y,
                eval_set=eval,
                verbose=False)
        
        pred_proba = clf.predict_proba(test_X)[:, 1]
        test_prec, test_recall, _ = precision_recall_curve(test_y, pred_proba)
        test_pr_auc = auc(test_recall, test_prec)

        praucs_test.append(test_pr_auc)

        pred_proba = clf.predict_proba(train_X)[:, 1]
        train_prec, train_recall, _ = precision_recall_curve(train_y, pred_proba)
        train_pr_auc = auc(train_recall, train_prec)

        praucs_train.append(train_pr_auc)

    print(' '.join([f'{p:.3f}' for p in praucs_test]) + " " + str(np.mean(praucs_test)) + " " + str(np.mean(praucs_train)))
    
    clf.fit(full_X, full_y)

    test_proba_y = clf.predict_proba(full_test)[:, 1]
    test_labels['SepsisLabel'] = test_proba_y
    test_labels['person_id_datetime'] = test_labels['person_id'].astype(str) + '_' + test_labels['datetime'].astype(str)
    test_labels[['person_id_datetime', 'SepsisLabel']].to_csv('preds.csv', index = False)

    api.competition_submit('preds.csv', str(params), 'phems-hackathon-early-sepsis-prediction')
        
    return {'loss':-np.mean(praucs_test), 'status': STATUS_OK}

In [32]:
trials = Trials()

best_hyperparams = fmin(fn = objective,
                        space = params,
                        algo = tpe.suggest,
                        max_evals = 99,
                        trials = trials)

0.523 0.653 0.503 0.468 0.739 0.5772356320892269 0.9519786549002067
  0%|          | 0/99 [03:58<?, ?trial/s, best loss=?]

  0%|          | 0.00/5.34M [00:00<?, ?B/s]
  2%|1         | 96.0k/5.34M [00:00<00:06, 871kB/s]
  7%|7         | 400k/5.34M [00:00<00:02, 2.04MB/s]
 11%|#1        | 608k/5.34M [00:00<00:03, 1.57MB/s]
 14%|#4        | 784k/5.34M [00:00<00:03, 1.28MB/s]
 17%|#6        | 928k/5.34M [00:00<00:03, 1.32MB/s]
 20%|#9        | 1.05M/5.34M [00:00<00:03, 1.36MB/s]
 22%|##2       | 1.19M/5.34M [00:00<00:03, 1.39MB/s]
 25%|##4       | 1.33M/5.34M [00:00<00:03, 1.40MB/s]
 28%|##7       | 1.48M/5.34M [00:01<00:02, 1.42MB/s]
 30%|###       | 1.62M/5.34M [00:01<00:02, 1.42MB/s]
 33%|###3      | 1.77M/5.34M [00:01<00:02, 1.42MB/s]
 36%|###5      | 1.91M/5.34M [00:01<00:02, 1.42MB/s]
 38%|###8      | 2.05M/5.34M [00:01<00:02, 1.43MB/s]
 41%|####      | 2.19M/5.34M [00:01<00:02, 1.43MB/s]
 44%|####3     | 2.33M/5.34M [00:01<00:02, 1.42MB/s]
 47%|####6     | 2.48M/5.34M [00:01<00:02, 1.44MB/s]
 49%|####9     | 2.64M/5.34M [00:01<00:01, 1.46MB/s]
 52%|#####2    | 2.78M/5.34M [00:02<00:01, 1.45MB/s]
 55%|##

0.514 0.736 0.525 0.456 0.692 0.5846405728893279 0.8677617906099611                 
  1%|          | 1/99 [09:50<7:22:04, 270.66s/trial, best loss: -0.5772356320892269]

  0%|          | 0.00/5.44M [00:00<?, ?B/s]
  2%|1         | 96.0k/5.44M [00:00<00:06, 827kB/s]
  7%|7         | 400k/5.44M [00:00<00:02, 2.01MB/s]
 11%|#         | 608k/5.44M [00:00<00:03, 1.67MB/s]
 14%|#4        | 784k/5.44M [00:00<00:03, 1.34MB/s]
 18%|#7        | 976k/5.44M [00:00<00:03, 1.32MB/s]
 22%|##1       | 1.19M/5.44M [00:00<00:02, 1.58MB/s]
 25%|##4       | 1.36M/5.44M [00:00<00:02, 1.54MB/s]
 28%|##7       | 1.52M/5.44M [00:01<00:02, 1.51MB/s]
 31%|###       | 1.67M/5.44M [00:01<00:02, 1.49MB/s]
 34%|###3      | 1.83M/5.44M [00:01<00:02, 1.47MB/s]
 36%|###6      | 1.98M/5.44M [00:01<00:02, 1.47MB/s]
 39%|###9      | 2.14M/5.44M [00:01<00:02, 1.46MB/s]
 42%|####1     | 2.28M/5.44M [00:01<00:02, 1.46MB/s]
 45%|####4     | 2.42M/5.44M [00:01<00:02, 1.45MB/s]
 47%|####7     | 2.56M/5.44M [00:01<00:02, 1.45MB/s]
 50%|####9     | 2.70M/5.44M [00:01<00:01, 1.45MB/s]
 52%|#####2    | 2.84M/5.44M [00:02<00:01, 1.44MB/s]
 55%|#####4    | 2.98M/5.44M [00:02<00:01, 1.44MB/s]
 57%|##

0.532 0.702 0.507 0.458 0.732 0.5862753021341112 0.9495594895470617                 
  2%|▏         | 2/99 [14:12<8:31:48, 316.59s/trial, best loss: -0.5846405728893279]

...

0.520 0.734 0.530 0.451 0.703 0.5878861399864943 0.9683862081074924                 
 98%|█████████▊| 97/99 [8:40:21<08:58, 269.30s/trial, best loss: -0.617738283624704]

  0%|          | 0.00/5.56M [00:00<?, ?B/s]
  2%|1         | 96.0k/5.56M [00:00<00:07, 803kB/s]
  7%|7         | 400k/5.56M [00:00<00:02, 2.02MB/s]
 11%|#         | 608k/5.56M [00:00<00:03, 1.41MB/s]
 13%|#3        | 768k/5.56M [00:00<00:04, 1.23MB/s]
 16%|#6        | 912k/5.56M [00:00<00:04, 1.08MB/s]
 20%|#9        | 1.11M/5.56M [00:00<00:03, 1.37MB/s]
 23%|##2       | 1.27M/5.56M [00:00<00:03, 1.39MB/s]
 26%|##5       | 1.42M/5.56M [00:01<00:03, 1.41MB/s]
 28%|##8       | 1.58M/5.56M [00:01<00:02, 1.41MB/s]
 31%|###       | 1.72M/5.56M [00:01<00:02, 1.42MB/s]
 34%|###3      | 1.88M/5.56M [00:01<00:02, 1.42MB/s]
 37%|###6      | 2.03M/5.56M [00:01<00:02, 1.44MB/s]
 39%|###9      | 2.17M/5.56M [00:01<00:02, 1.44MB/s]
 42%|####1     | 2.31M/5.56M [00:01<00:02, 1.45MB/s]
 44%|####4     | 2.45M/5.56M [00:01<00:02, 1.44MB/s]
 47%|####6     | 2.59M/5.56M [00:01<00:02, 1.42MB/s]
 49%|####9     | 2.75M/5.56M [00:02<00:02, 1.44MB/s]
 52%|#####2    | 2.91M/5.56M [00:02<00:01, 1.45MB/s]
 55%|##

0.532 0.744 0.524 0.475 0.726 0.6003221115681073 0.9522401321371351                 
 99%|█████████▉| 98/99 [8:43:10<04:25, 265.89s/trial, best loss: -0.617738283624704]

  0%|          | 0.00/5.40M [00:00<?, ?B/s]
  2%|1         | 96.0k/5.40M [00:00<00:06, 834kB/s]
  7%|7         | 400k/5.40M [00:00<00:02, 2.03MB/s]
 11%|#1        | 608k/5.40M [00:00<00:03, 1.54MB/s]
 14%|#4        | 784k/5.40M [00:00<00:03, 1.23MB/s]
 17%|#6        | 928k/5.40M [00:00<00:04, 1.14MB/s]
 21%|##        | 1.12M/5.40M [00:00<00:03, 1.37MB/s]
 24%|##3       | 1.28M/5.40M [00:00<00:03, 1.39MB/s]
 27%|##6       | 1.44M/5.40M [00:01<00:02, 1.41MB/s]
 30%|##9       | 1.59M/5.40M [00:01<00:02, 1.43MB/s]
 32%|###2      | 1.75M/5.40M [00:01<00:02, 1.42MB/s]
 35%|###5      | 1.91M/5.40M [00:01<00:02, 1.43MB/s]
 38%|###7      | 2.05M/5.40M [00:01<00:02, 1.44MB/s]
 41%|####      | 2.19M/5.40M [00:01<00:02, 1.44MB/s]
 43%|####3     | 2.34M/5.40M [00:01<00:02, 1.43MB/s]
 46%|####6     | 2.50M/5.40M [00:01<00:02, 1.45MB/s]
 49%|####8     | 2.64M/5.40M [00:01<00:01, 1.45MB/s]
 52%|#####1    | 2.78M/5.40M [00:02<00:01, 1.45MB/s]
 54%|#####4    | 2.92M/5.40M [00:02<00:01, 1.43MB/s]
 57%|##

100%|██████████| 99/99 [8:43:28<00:00, 317.26s/trial, best loss: -0.617738283624704]


In [33]:
best_hyperparams

{'colsample_bytree': np.float64(0.7064703971573795),
 'gamma': np.float64(1.3641799557272378),
 'learning_rate': np.float64(0.02661060698481256),
 'max_depth': np.float64(10.0),
 'min_child_weight': np.float64(9.009682994381349),
 'n_estimators': np.float64(150.0),
 'reg_alpha': np.float64(0.33992901832205574),
 'reg_lambda': np.float64(0.11730507357393373),
 'subsample': np.float64(0.6597272051775671)}

In [34]:
for k, v in best_hyperparams.items():
    print(f"{k} = {v},")

colsample_bytree = 0.7064703971573795,
gamma = 1.3641799557272378,
learning_rate = 0.02661060698481256,
max_depth = 10.0,
min_child_weight = 9.009682994381349,
n_estimators = 150.0,
reg_alpha = 0.33992901832205574,
reg_lambda = 0.11730507357393373,
subsample = 0.6597272051775671,
